# Repetition Code Syndrome Extraction

This notebook demonstrates stabilizer simulation of syndrome extraction for the **repetition code** using the `paulimer` library.

The repetition code encodes a single logical qubit in $n$ physical data qubits. Its stabilizers are $Z_i Z_{i+1}$ for adjacent qubit pairs. Measuring these stabilizers (syndrome extraction) detects bit-flip errors without collapsing the logical state.

## Setup

In [1]:
from paulimer import OutcomeCompleteSimulation
from paulimer import UnitaryOpcode
from paulimer import SparsePauli

## Quantum Primitives

Helper functions wrapping basic operations: Hadamard, CNOT, Bell pair preparation, and Pauli measurements. The `measure_zz_via_cnot` function implements $ZZ$ measurement using two CNOTs and an auxiliary qubit.

In [2]:
def h(sim, target):
    sim.apply_unitary(unitary_op= UnitaryOpcode.Hadamard, support=[target])

def cx(sim, control, target):
    sim.apply_unitary(unitary_op= UnitaryOpcode.ControlledX, support=[control, target])

def prepare_bell_pair(sim, qubit_a, qubit_b):
    sim.apply_unitary(unitary_op= UnitaryOpcode.PrepareBell, support=[qubit_a, qubit_b])

def prepare_bell_pairs(sim, qubits_a, qubits_b):
    for (a,b) in zip(qubits_a,qubits_b):
        prepare_bell_pair(sim, a,b)

def m_reset_z(sim, target):
    bit = sim.measure(SparsePauli.z(target))
    sim.apply_conditional_pauli(SparsePauli.x(target), [bit])
    return bit

def zz(qubit_a, qubit_b):
    return SparsePauli.z(qubit_a) * SparsePauli.z(qubit_b)

def measure_zz(sim, qubit_a, qubit_b):
    bit = sim.measure(zz(qubit_a, qubit_b))
    return bit

def xx(qubit_a, qubit_b):
    return SparsePauli.x(qubit_a) * SparsePauli.x(qubit_b)

def measure_zz_via_cnot(sim, qubit_a, qubit_b, aux):
    """Measure ZZ using two CNOTs and a single auxiliary qubit."""
    cx(sim, qubit_a, aux)
    cx(sim, qubit_b, aux)
    bit = m_reset_z(sim, aux)
    return [bit]

## Syndrome Extraction Circuit

Extracts all $n-1$ stabilizer syndromes using a two-layer schedule:
- **Odd layer**: measures $Z_0 Z_1$, $Z_2 Z_3$, ... (non-overlapping pairs)
- **Even layer**: measures $Z_1 Z_2$, $Z_3 Z_4$, ... (interleaved pairs)

In [3]:
def repetition_code_syndrome_extraction(sim, data_qubits, aux_qubits):
    num_stabilizers = len(data_qubits) - 1
    
    odd_syndrome = []
    # Layer using odd aux qubits: measure ZZ on pairs (0,1), (2,3), ...
    for i in range(0, num_stabilizers, 2):
        bit = measure_zz_via_cnot(sim, data_qubits[i], data_qubits[i + 1], aux_qubits[i])
        odd_syndrome.append(bit)
    
    even_syndrome = []
    # Layer using even aux qubits: measure ZZ on pairs (1,2), (3,4), ...
    for i in range(1, num_stabilizers, 2):
        bit = measure_zz_via_cnot(sim, data_qubits[i], data_qubits[i + 1], aux_qubits[i])
        even_syndrome.append(bit)
    
    # Interleave odd and even syndrome measurements to get syndromes in order
    syndrome = []
    for i in range(num_stabilizers):
        if i % 2 == 0:
            syndrome.append(odd_syndrome[i // 2])
        else:
            syndrome.append(even_syndrome[i // 2])
    
    return syndrome

## Verification via Bell Pairs

To verify correctness, we entangle each data qubit with a reference qubit via Bell pairs *before* syndrome extraction. After extraction, `is_stabilizer` checks that each $Z_i Z_{i+1}$ is indeed a stabilizer with the measured syndrome sign. This confirms the circuit correctly projects onto the stabilizer eigenspace.

In [4]:
def test_syndrome_extraction(sim, data_qubit_count: int):
    num_stabilizers = data_qubit_count - 1
    data_qubits = list(range(data_qubit_count))
    reference_qubits = list(range(data_qubit_count, 2 * data_qubit_count))
    aux_qubits = list(range(2 * data_qubit_count, 3 * data_qubit_count - 1))
    prepare_bell_pairs(sim, data_qubits, reference_qubits)
    syndrome = repetition_code_syndrome_extraction(sim, data_qubits, aux_qubits)

    for q in aux_qubits:
        sim.is_stabilizer(SparsePauli.z(q))

    for i in range(num_stabilizers):
        sim.is_stabilizer(zz(data_qubits[i], data_qubits[i + 1]), sign_parity=syndrome[i])



## Run Tests

Test syndrome extraction for repetition codes of sizes 3 to 199. Each test creates a fresh simulation, runs the verification, and implicitly asserts correctness via `is_stabilizer`.

In [5]:
for n in range(3, 200):
    sim = OutcomeCompleteSimulation()
    test_syndrome_extraction(sim, n)